## The model 6 about Metrics for Classification model and Multiclassification

## homework by matrix error and metrics for classification

In [225]:
import pandas as pd
import numpy as np

In [226]:
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [227]:
from sklearn.metrics import confusion_matrix, \
                            recall_score, \
                            accuracy_score

In [228]:
titanic = pd.read_csv('train.csv')
print('\nForm:', titanic.shape)
print('\nName_features:', titanic.columns)
print('\nType:', titanic.dtypes)
titanic.head(10)


Form: (891, 12)

Name_features: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Type: PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Check data for NULL values

In [229]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [230]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [231]:
def preprocess_data(data):
    # fill all fills with none in column age
    mean_age_sex = (data
                    .groupby('Sex')
                    .agg({'Age': 'mean'}))

    mean_age_condition = np.where(data['Age'].isnull() == True,
                                  mean_age_sex.loc[data['Sex'], 'Age'],
                                  data['Age'])
    data['Age'] = mean_age_condition
    
    # fill all with none columns cabin embarked
    data.fillna('U', inplace=True)
    data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
    data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
    data['Embarked'].replace(['U', 'S', 'C', 'Q'], [0, 1, 2, 3], inplace=True)
    
    return data

In [232]:
titanic = preprocess_data(titanic)

In [233]:
y = titanic['Survived']
titanic.drop(['Survived'], axis=1, inplace=True)

In [234]:
x_train, x_valid, y_train, y_valid = train_test_split(titanic, y, random_state=17)

In [235]:
x_train.shape, x_valid.shape

((668, 7), (223, 7))

In [236]:
y_train.value_counts()

0    418
1    250
Name: Survived, dtype: int64

In [237]:
y_test = pd.read_csv('gender_submission.csv')
x_test = preprocess_data(pd.read_csv('test.csv'))

In [238]:
y_test.drop('PassengerId', axis=1, inplace=True)

In [239]:
y_test.head()

,Survived
0,0
1,1
2,0
3,0
4,1


In [240]:
knn_cls = KNeighborsClassifier(n_neighbors=5).fit(x_train, y_train)
naive_cls = GaussianNB().fit(x_train, y_train)
tree_cls = DecisionTreeClassifier().fit(x_train, y_train)
log_cls = LogisticRegression(max_iter=200).fit(x_train, y_train)

In [241]:
print(x_valid.shape)
x_valid.head()

(223, 7)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
890,3,0,32.0,0,0,7.7500,3
498,1,1,25.0,1,2,151.5500,1
664,3,0,20.0,1,0,7.9250,1
261,3,0,3.0,4,2,31.3875,1
716,1,1,38.0,0,0,227.5250,2


In [242]:
print(x_test.shape)
x_test['Embarked'].unique()

(418, 7)


array([3, 1, 2])

In [248]:
x_test.head(50)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.500000,0,0,7.8292,3
1,3,1,47.000000,1,0,7.0,1
2,2,0,62.000000,0,0,9.6875,3
3,3,0,27.000000,0,0,8.6625,1
4,3,1,22.000000,1,1,12.2875,1
5,3,0,14.000000,0,0,9.225,1
6,3,1,30.000000,0,0,7.6292,3
7,2,0,26.000000,1,1,29.0,1
8,3,1,18.000000,0,0,7.2292,2
9,3,0,21.000000,2,0,24.15,1


In [243]:
pred_knn = knn_cls.predict(x_valid)
pred_naive = naive_cls.predict(x_valid)
pred_tree = tree_cls.predict(x_valid)
pred_log = log_cls.predict(x_valid)

test_pred_knn = knn_cls.predict(x_test)
test_pred_naive = naive_cls.predict(x_test)
test_pred_tree = tree_cls.predict(x_test)
test_pred_log = log_cls.predict(x_test)

ValueError: could not convert string to float: 'U'

In [63]:
# matrix confusion for K-nearest neighbor 
print(confusion_matrix(y_valid, pred_knn))
print(f'knn recall: {recall_score(y_valid, pred_knn)}')
print(f'knn test recall: {recall_score(y_valid, pred_knn)}')
print(f'knn accuracy: {accuracy_score(y_valid, pred_knn)}')
print(f'knn test accuracy: {accuracy_score(y_valid, pred_knn)}')

[[114  17]
 [ 40  52]]
knn recall: 0.5652173913043478
knn accuracy: 0.7443946188340808


In [65]:
# matrix confusion for naive bayse classifier
print(confusion_matrix(y_valid, pred_naive))
print(f'naive recall: {recall_score(y_valid, pred_naive)}')
print(f'knn accuracy: {accuracy_score(y_valid, pred_naive)}')

[[131   0]
 [  0  92]]
naive recall: 1.0
knn accuracy: 1.0


In [66]:
# matrix confusion for decision tree
print(confusion_matrix(y_valid, pred_tree))
print(f'tree recall: {recall_score(y_valid, pred_tree)}')
print(f'tree accuracy: {accuracy_score(y_valid, pred_tree)}')

[[131   0]
 [  0  92]]
tree recall: 1.0
tree accuracy: 1.0


In [67]:
# matrix confusion for logistic regression
print(confusion_matrix(y_valid, pred_tree))
print(f'log recall: {recall_score(y_valid, pred_log)}')
print(f'log accuracy: {accuracy_score(y_valid, pred_log)}')

[[131   0]
 [  0  92]]
log recall: 1.0
log accuracy: 1.0
